In [1]:
import os

In [2]:
import googleapiclient.discovery

In [20]:
def google_api(id):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    # os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = "generate your own key and paste here."

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part="id,snippet",
        maxResults=300,
        order="relevance",
#         videoId= "ZZRP70zMHgo" #pos
        videoId= "x98h_BmY8-M" #neg
    )
    
    response = request.execute()

#     print(response)
    return response
response = google_api("621oD2zBSbI")

In [22]:
import pandas as pd
def create_df_author_comments():
    authorname = []
    comments = []
    for i in range(len(response["items"])):
        authorname.append(response["items"][i]["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"])
        comments.append(response["items"][i]["snippet"]["topLevelComment"]["snippet"]["textOriginal"])
    df_1 = pd.DataFrame(comments, index = authorname,columns=["Comments"])
    return df_1 
df_1 = create_df_author_comments()
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
# df_1

In [23]:
# REMOVE NaN VALUES AND EMPTY STRINGS:
df_1.dropna(inplace=True)

blanks = []  # start with an empty list

for i, comments in df_1.itertuples():  
    if type(comments)==str:            
        if comments.isspace():        
            blanks.append(i)     

df_1.drop(blanks, inplace=True)

In [24]:
# adding score to data frame
# import nltk
# nltk.download('vader_lexicon') 

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# df_1= df_1.assign(Score = lambda x: sid.polarity_scores('Comments'))
df_1['Scores'] = df_1['Comments'].apply(lambda x: sid.polarity_scores(x))

#normalize the score
df_1['Compound'] = df_1['Scores'].apply(lambda score_dict: score_dict['compound'])

#label
df_1['Polarity'] = df_1['Compound'].apply(lambda c: 'neutral' if -0.05 < c < 0.05 else ('negative' if c < -0.05 else 'positive'))

In [25]:
#evaluate polarity without stop words

In [26]:
#remove the stop words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop = stopwords.words('english')
df_1['comments (without stopwords)'] = df_1['Comments'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
# df_1.sort_values(by=['Polarity']).head()

In [27]:
#convert to lower case
lower = lambda x: x.lower()
df_1['comments (without stopwords)'] = df_1['comments (without stopwords)'].apply(lower)  

In [28]:
#remove emojis and special characters
import re
def cleaning_comments(comment):
  comment = re.sub("[🤣|🤭|🤣|😁|🤭|❤️|👍|🏴|😣|😠|💪|🙏|😞|🌺|🌸|🌞|🌻|💐|💓|😥|💔|😪|😑|🏽|😢|😑|😇|💜|🪴|🙌🏻|🇨🇦|🕊|🕯|😭|😔|💙|🏼|✝|🇿]+",'',comment)
  comment = re.sub("[0-9]+","",comment)
  comment = re.sub("[\:|\@|\)|\*|\.|\$|\!|\?|\,|\%|\"|\(|\-|\”|\#|\!|\/|\«|\»|\&|\n|\'|\;|\!|<|>|\'|\’|\\\\]+"," ",comment)
  return comment

df_1['comments (without stopwords)']= df_1['comments (without stopwords)'].apply(cleaning_comments)

In [29]:
# adding score to data frame
# import nltk
# nltk.download('vader_lexicon') 

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# df_1= df_1.assign(Score = lambda x: sid.polarity_scores('Comments'))
df_1['scores'] = df_1['comments (without stopwords)'].apply(lambda x: sid.polarity_scores(x))

#normalize the score
df_1['compound'] = df_1['scores'].apply(lambda score_dict: score_dict['compound'])

#label
df_1['polarity (without stopwords)'] = df_1['compound'].apply(lambda c: 'neutral' if -0.05 < c < 0.05 else ('negative' if c < -0.05 else 'positive'))

In [30]:
df_1['Polarity'].value_counts()

negative    68
positive    24
neutral      8
Name: Polarity, dtype: int64

In [31]:
df_1['polarity (without stopwords)'].value_counts()

negative    65
positive    24
neutral     11
Name: polarity (without stopwords), dtype: int64

In [32]:
# df_1.loc[(df_1['Polarity'] == 'negative') & (df_1['polarity'] == 'positive')]
df_1.loc[(df_1['Polarity'] != df_1['polarity (without stopwords)'])]

,Comments,Scores,Compound,Polarity,comments (without stopwords),scores,compound,polarity (without stopwords)
DwayneStorm,My heart aches so much for those parents that brought their children to school but didn't get a chance to bring them home 😞,"{'neg': 0.153, 'neu': 0.847, 'pos': 0.0, 'compound': -0.3839}",-0.3839,negative,my heart aches much parents brought children school get chance bring home,"{'neg': 0.143, 'neu': 0.714, 'pos': 0.143, 'compound': 0.0}",0.0000,neutral
- madi -,my heart goes out to the parents who took their kids to school but did not take them home </3\n\n\n\nRIP,"{'neg': 0.216, 'neu': 0.784, 'pos': 0.0, 'compound': -0.7579}",-0.7579,negative,heart goes parents took kids school take home rip,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral
I Rod,"My heart goes out to the families of the innocent lives lost in the tragedy may GOD take them UP!! Parents and families who read this and all over….ACTUALLY PARENT YOUR CHILDREN PROPERLY!!! Be in there loves and involved in all there actions especially if they live under your rooftop!!! Don’t let your children kill innocent people and children!!! Stop just going through the motions and raise the children with morals and principles so they know right from wrong!! Send them into the world as good people so that the world doesn’t have to suffer their evil!! Sup it before it leaves the doors, it all starts at home!!! DO NOT ABUSE THEM IN ANY WAY and DO NOT EXPOSE them to negativity!! Explain what the negatives can lead to in their lives! Show them how remaining positive and with your support, no matter what they’ll over come!!","{'neg': 0.166, 'neu': 0.687, 'pos': 0.147, 'compound': -0.7743}",-0.7743,negative,my heart goes families innocent lives lost tragedy may god take up parents families read over… actually parent your children properly be loves involved actions especially live rooftop don t let children kill innocent people children stop going motions raise children morals principles know right wrong send world good people world doesn t suffer evil sup leaves doors starts home do not abuse them in any way do not expose negativity explain negatives lead lives show remaining positive support matter they ll come,"{'neg': 0.212, 'neu': 0.543, 'pos': 0.245, 'compound': -0.048}",-0.0480,neutral
Christian Delgado,I can’t believe this monsters mom tried to say “ don’t come at him he did it for a reason “. You do not kill your own family member AND CHILDREN ALONGSIDE INNOCENT TEACHERS. There is NO excuse for that.,"{'neg': 0.067, 'neu': 0.748, 'pos': 0.185, 'compound': 0.6414}",0.6414,positive,i can t believe monsters mom tried say “ don t come reason “ you kill family member and children alongside innocent teachers there no excuse that,"{'neg': 0.241, 'neu': 0.629, 'pos': 0.129, 'compound': -0.6369}",-0.6369,negative
Askingoldman,Sad but true. Won’t be the last. Tragedies don’t get laws change. We live in a country where all lives don’t matter.,"{'neg': 0.213, 'neu': 0.613, 'pos': 0.175, 'compound': -0.2617}",-0.2617,negative,sad true won t last tragedies don t get laws change we live country lives don t matter,"{'neg': 0.254, 'neu': 0.424, 'pos': 0.322, 'compound': 0.1531}",0.1531,positive


In [33]:
df_1.loc[(df_1['Polarity']=='negative') | (df_1['polarity (without stopwords)']=='negative')]

,Comments,Scores,Compound,Polarity,comments (without stopwords),scores,compound,polarity (without stopwords)
DwayneStorm,My heart aches so much for those parents that brought their children to school but didn't get a chance to bring them home 😞,"{'neg': 0.153, 'neu': 0.847, 'pos': 0.0, 'compound': -0.3839}",-0.3839,negative,my heart aches much parents brought children school get chance bring home,"{'neg': 0.143, 'neu': 0.714, 'pos': 0.143, 'compound': 0.0}",0.0000,neutral
Matthew Hagner,Turns out that no officer ever went in to the school. This whole situation is disgusting and my heart breaks for all who lost their lives and the families impacted,"{'neg': 0.226, 'neu': 0.774, 'pos': 0.0, 'compound': -0.7845}",-0.7845,negative,turns officer ever went school this whole situation disgusting heart breaks lost lives families impacted,"{'neg': 0.305, 'neu': 0.695, 'pos': 0.0, 'compound': -0.6908}",-0.6908,negative
AGENTBOOOM,It always amazes me how people don't disclose information about individuals odd behaviors or threats they make to innocent people until after a massacre like this happens. People need to learn to speak up as this tragedy could've possibly be avoided.,"{'neg': 0.237, 'neu': 0.613, 'pos': 0.15, 'compound': -0.6858}",-0.6858,negative,it always amazes people disclose information individuals odd behaviors threats make innocent people massacre like happens people need learn speak tragedy could ve possibly avoided,"{'neg': 0.313, 'neu': 0.474, 'pos': 0.213, 'compound': -0.5859}",-0.5859,negative
Ingrid wynne,"My heart goes out to the families of the murdered children and teachers. It's tragic, but America must address its gun laws","{'neg': 0.291, 'neu': 0.709, 'pos': 0.0, 'compound': -0.7783}",-0.7783,negative,my heart goes families murdered children teachers it s tragic america must address gun laws,"{'neg': 0.471, 'neu': 0.529, 'pos': 0.0, 'compound': -0.8689}",-0.8689,negative
Ariel Perez,I can’t believe people would be willing to take the lives of innocent kids for no reason. I feel so bad for what the families are going through,"{'neg': 0.209, 'neu': 0.717, 'pos': 0.075, 'compound': -0.6476}",-0.6476,negative,i can t believe people would willing take lives innocent kids reason i feel bad families going,"{'neg': 0.196, 'neu': 0.67, 'pos': 0.134, 'compound': -0.2732}",-0.2732,negative
Rob Flanagan,"I have been following this story since it unfolded from the very beginning. I’m in my late forties, and a working professional from the Boston area. I don’t have kids, and this is the hardest thing for me to understand and make sense of it all. My heart goes out to the parents - single parents, dual parents that brought their children to school to drop them off for classes. I see the images as those young and innocent children cut down by a mad man, with yet another senseless tale of unnecessary violence & gunfire with automatic weapons. When is this going to end? Forget about whose to blame on the scene. From the police officers that immediately went into the building who the public finds out later was fabricated and not true, to the officer that is guarding both the main entrance of the school and walkway is now a ghost. While finger pointing continues, a quiet community is rocked and shaken by this senseless tragedy. When does this end. We as a society have a serious epidemic of senseless violence in America with these long barrel automatic weapons getting into the hands of minors, young teenagers, and those with mental health issues, and criminal records. President Biden, get off your ass and finally do something that no President has done before and end this madness. We need to end this by having Congress exact strict measures and laws implementing the banning of minors, teens, criminals from getting their hands on military assault weapons. There is absolutely no need for this. I don’t have to be a parent to be emotional and know that something needs to be done now. Stop finger pointing and blaming when the real pr

In [19]:
df_1.sort_values(by=['Polarity'])

,Comments,Scores,Compound,Polarity,comments (without stopwords),scores,compound,polarity (without stopwords)
Moongazey Cat,Isn't this just adorable? She is such a sport 😀💜💜,"{'neg': 0.247, 'neu': 0.753, 'pos': 0.0, 'compound': -0.3875}",-0.3875,negative,isn t adorable she sport 😀,"{'neg': 0.0, 'neu': 0.484, 'pos': 0.516, 'compound': 0.4939}",0.4939,positive
CaNNeD CRaZy,"""Thank you ma'am.....for everything!!!""\n\nShe is living history! When she goes the world will never see anyone else like her again. It will be devastating.","{'neg': 0.132, 'neu': 0.678, 'pos': 0.19, 'compound': -0.3544}",-0.3544,negative,thank ma am for everything she living history when goes world never see anyone else like again it devastating,"{'neg': 0.17, 'neu': 0.632, 'pos': 0.198, 'compound': -0.0772}",-0.0772,negative
Jamie W,I got tears in my eyes when i watched this.,"{'neg': 0.213, 'neu': 0.787, 'pos': 0.0, 'compound': -0.2263}",-0.2263,negative,i got tears eyes watched this,"{'neg': 0.322, 'neu': 0.678, 'pos': 0.0, 'compound': -0.2263}",-0.2263,negative
Malthizar,"If anyone deserves to have an audience with the Queen, it's Paddington","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral,if anyone deserves audience queen paddington,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral
liouy cnny,"Absolutely priceless , way too short, I could have watched Elizabeth and Paddington for much longer, my favourite part of the whole 4 days.","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral,absolutely priceless way short i could watched elizabeth paddington much longer favourite part whole days,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral
OAKWOOD FARM,"Absolutely priceless , way too short, I could have watched Elizabeth and Paddington for much longer, my favourite part of the whole 4 days.","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral,absolutely priceless way short i could watched elizabeth paddington much longer favourite part whole days,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral
silou,The Queen should win an Academy Award for this...natural Born actress,"{'neg': 0.0, 'neu': 0.552, 'pos': 0.448, 'compound': 0.8074}",0.8074,positive,the queen win academy award this natural born actress,"{'neg': 0.0, 'neu': 0.371, 'pos': 0.629, 'compound': 0.88}",0.8800,positive
Ivy\t 🔥𝐆𝐨 𝐓𝐨 𝐌𝐲 𝐂𝐡𝐚𝐧𝐧𝐞𝐥 [𝐋!𝐯𝐞],This is truly adorable. The Queen is hilarious in her regal and proper manner.,"{'neg': 0.0, 'neu': 0.556, 'pos': 0.444, 'compound': 0.8316}",0.8316,positive,this truly adorable the queen hilarious regal proper manner,"{'neg': 0.0, 'neu': 0.405, 'pos': 0.595, 'compound': 0.8316}",0.8316,positive
Caitie,This made me tear up a little. I love the Queen and I'm not even British,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.6003}",0.6003,positive,this made tear little i love queen i m even british,"{'neg': 0.0, 'neu': 0.642, 'pos': 0.358, 'compound': 0.6003}",0.6003,positive
Duchess_of_Petty,Paddington is the most adorable thing.,"{'neg': 0.0, 'neu': 0.589, 'pos': 0.411, 'compound': 0.5413}",0.5413,positive,paddington adorable thing,"{'neg': 0.0, 'neu': 0.385, 'pos': 0.615, 'compound': 0.4939}",0.4939,positive
